<a href="https://colab.research.google.com/github/Xixaus/Nastroje/blob/main/CD_extrakce%2Bvyhlazen%C3%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pro spuštění skriptu je nutné se přihlásit do Google účtu.
2. Spusťte 1. buňku tlačítkem ▶ v levém horním rohu nebo kliknutím do buňky a stistnutím klávesové zkratky ctrl+enter aby se načetly knihovny a vytvořili složky.
3. Do složky Samples vložte soubory s vzorky a do složky blanks vložte soubory s blanky

    !!! Je nutné aby pH vzoku bylo v názvu v souboru v následujícím formátu: ... _ 7_05_pufr.txt, kdy tímto způsobem bude končit název souboru !!!

3. Spusťte 2. buňku pro vygenerování grafu a souboru
    
    !!! pokud !!!
4. Soubor data.txt vložte do Bestselu, pokud Vám bude hlásit chybu, zkopírujte všechny data ze souboru a vložte do okna v Bestselu

In [1]:
#@title Vstupní hodnoty { run: "auto", vertical-output: true }

file_version = "new" # @param ["old", "new"] {allow-input: true}


In [5]:
#@title Knihovny a vytvoření/smazání složek { run: "auto", vertical-output: true }

#načtení knihoven
import os
import shutil
import pandas as pd
from scipy.signal import savgol_filter
import plotly.graph_objects as go

#vytvoření potřebných složek pro data
if os.path.exists("/content/samples"):
    shutil.rmtree("/content/samples")
os.makedirs("/content/samples", exist_ok=True)

if os.path.exists("/content/blanks"):
    shutil.rmtree("/content/blanks")
os.makedirs("/content/blanks", exist_ok=True)

#smazání zbytečné složky
if os.path.exists("/content/sample_data"):
    shutil.rmtree("/content/sample_data")

In [21]:
#@title Výpočty { run: "auto", vertical-output: true }

#funkce pro extrakci dat ze souborů

def extract_data(folder_path, file_version = "old"):
    file_list = os.listdir(folder_path)
    df = pd.DataFrame() #vytvoření prázného dataframe

    for filename in file_list: #iterace každého souboru ve složce
        if filename.endswith(".txt"): #podmínka pro extrakci pouze z .txt souborů
            number = (filename.split("_")[-3].split(".")[0]) + "." + filename.split("_")[-2].split(".")[0] #pojmenování spektra, konkrétně pH, které je na konci ve formátu ..._7_05_pufr.txt

            if file_version == "old":
                axis = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[0], skiprows=19, header=None) #extrakce vlnových délek
                axis = axis.values.flatten()

                data = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[1], skiprows=19, header=None) # extrakce spektra
                data = data.rename(columns={1: float(number)}) #přejmenování spektra podle promněnné number
                data.index = axis

            else:
                axis = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[0], skiprows=20, header=None) #extrakce vlnových délek
                end = axis[axis == "##### Extended Information"].dropna(how='all').index.item()
                axis = axis.iloc[:end,].values.flatten()

                data = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[1], skiprows=20, header=None) # extrakce spektra
                data = data.iloc[:end,]
                data = data.rename(columns={1: float(number)}) #přejmenování spektra podle promněnné number
                data.index = axis

            df = pd.concat([df, data], axis=1) #spojení dataframe se spectrem

    df = df.sort_index(axis=1) #seřazení sepekter podle názvu (number)
    df = df.applymap(lambda x: float(x.replace(',', '.')))
    df.index = df.index.to_series().apply(lambda x: float(x.replace(',', '.')))
    return(df)

samples = extract_data("/content/samples", file_version) #extrakce naměřených spekter
blanks = extract_data("/content/blanks", file_version) #extrakce blanků


spectra = samples - blanks
spectra_SG = samples - blanks #odečtení blanků od spekter

spectra = spectra.astype("float16")
spectra_SG = spectra_SG.astype("float16")

for column in spectra_SG.columns:
    spectra_SG[column] = savgol_filter(spectra_SG[column], window_length=7, polyorder=2, deriv=0) #vyhlazení pomocí savitzkého-golayových filtrů

fig1 = go.Figure() #vykreslení grafu

for column in spectra.columns:
    fig1.add_trace(go.Scatter(y=spectra[column], x=spectra.index, mode='lines', name=str(column)))

fig1.update_layout(title="Spektra")
fig1.show()

fig2 = go.Figure() #vykreslení grafu
display(spectra)


spectra.to_csv('spectra.txt', sep='\t') #uložení spekter do .txt souboru
spectra_SG.to_csv('spectra_vyhlazena.txt', sep='\t') #uložení spekter do .txt souboru

,3.05,4.05,5.05,6.05,7.05
350.0,-0.008820,-0.027649,0.048401,0.018158,0.085449
349.0,-0.003206,-0.002338,0.028214,0.049072,0.000797
348.0,0.026276,-0.020950,0.002590,-0.009926,0.017426
347.0,-0.006084,-0.011215,-0.027985,0.015640,0.066589
346.0,-0.010490,-0.006630,0.050476,0.018448,0.021912
...,...,...,...,...,...
204.0,1.295898,0.862305,0.936035,1.426758,0.717285
203.0,0.650391,0.315186,0.427979,0.405273,0.792480
202.0,0.141846,0.377441,0.680176,1.101562,0.704590
201.0,0.561523,0.943359,0.313232,0.971680,1.208008


In [22]:
for column in spectra.columns:
    fig2.add_trace(go.Scatter(y=spectra_SG[column], x=spectra_SG.index, mode='lines', name=str(column)))

fig2.update_layout(title="Vyhlazená spektra")
fig2.show()

spectra_SG.astype("float16")

display(spectra_SG)

,3.05,4.05,5.05,6.05,7.05
350.0,0.001490,-0.018576,0.046525,0.030306,0.066549
349.0,-0.001090,-0.016949,0.020200,0.019411,0.039885
348.0,-0.003541,-0.015840,0.007361,0.012779,0.023696
347.0,-0.005863,-0.015250,0.008010,0.010410,0.017980
346.0,-0.013238,-0.020698,0.038898,0.008198,0.037025
...,...,...,...,...,...
204.0,0.843064,0.347479,0.640180,0.670625,0.519938
203.0,0.778774,0.491178,0.583374,0.790315,0.896717
202.0,0.645203,0.619095,0.724165,1.055198,0.912458
201.0,0.422634,0.752415,0.903229,1.379412,0.771188
